# Consistency Check Agent 테스트 (Production Level)

일관성 검사 에이전트 (Level 2) 테스트 노트북

## 역할: "Story Consistency Expert" (스토리 일관성 전문가)
- 캐릭터, 이벤트, 관계 간 충돌 감지
- Dialogue/Emotion 데이터 교차 검증
- 프로그래매틱 백업 검증 (trait 모순, 방향성 오류)
- **자동 해결 전략 (suggested_action) 제공**
- **🆕 수정 제안값 (final_value_candidate) 제공**

> **Note**: 모든 Level 1 Agent의 출력을 통합하여 일관성을 검사합니다.

---

### 💡 충돌 유형 (Conflict Types)

| 충돌 유형 | Severity | suggested_action |
|----------|----------|------------------|
| **CHARACTER_TRAIT_CONFLICT** | HIGH | FLAG_FOR_HUMAN |
| **DIRECTION_CONFLICT** | MEDIUM | AUTO_FIX |
| **REFERENTIAL_INTEGRITY_ERROR** | HIGH | FLAG_FOR_HUMAN |

### 🔧 Auto-Resolution Actions

| Action | 설명 | final_value_candidate |
|--------|------|----------------------|
| `AUTO_FIX` | 시스템 자동 수정 | `{table, key, update}` - 바로 UPDATE 가능 |
| `FLAG_FOR_HUMAN` | 인간 검토 필요 | `{options, requires_selection}` 또는 `null` |
| `OVERWRITE_WITH_NEW` | 새 값으로 덮어쓰기 | 새 추출값 구조 |

In [1]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [2]:
SAMPLE_STORY = """
서진과 이민호는 한때 가장 친한 친구였다. 어릴 때부터 함께 검술을 배우며 자랐다.
하지만 이민호가 마을을 배신한 후, 그들은 적이 되었다.

하나는 서진의 오랜 동료이자 마을의 치료사였다. 그녀는 항상 서진을 걱정했다.
"""

EXTRACTED_CHARACTERS = [
    {"name": "서진", "role": "protagonist", "status": "alive",
     "personality": {"core_traits": ["brave", "skilled", "loyal"]}, "visual": {}},
    {"name": "하나", "role": "supporting", "status": "alive",
     "personality": {"core_traits": ["wise", "calm", "caring"]}, "visual": {}},
    {"name": "이민호", "role": "antagonist", "status": "alive",
     "personality": {"core_traits": ["bitter", "angry", "skilled"]}, "visual": {}}
]

RELATIONSHIP_GRAPH = {
    "relationships": [
        {"source": "이민호", "target": "서진", "relation_type": "BETRAYED", "strength": 9, 
         "bidirectional": False, "evolved_from": "FRIENDLY"},
        {"source": "서진", "target": "하나", "relation_type": "FRIENDLY", "strength": 8,
         "bidirectional": True}
    ]
}

ANALYZED_DIALOGUES = {"speech_patterns": []}
TRACKED_EMOTIONS = {"emotion_states": []}

def create_base_state(story=SAMPLE_STORY):
    return {
        "content": story, "project_id": "test-001", "document_id": "doc-001", 
        "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
        "extracted_characters": EXTRACTED_CHARACTERS,
        "extracted_events": [],
        "relationship_graph": RELATIONSHIP_GRAPH,
        "analyzed_dialogues": ANALYZED_DIALOGUES,
        "tracked_emotions": TRACKED_EMOTIONS,
        "consistency_report": {}, "messages": [], "errors": []
    }

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## 1. 방향성 오류 테스트 - final_value_candidate 확인

In [3]:
from app.agents.analysis.consistency import consistency_check_node, SuggestedAction

async def test_direction_conflict():
    print("🧭 Direction Conflict 테스트 (final_value_candidate 확인)...")
    state = create_base_state()
    state["relationship_graph"] = {
        "relationships": [
            {"source": "이민호", "target": "서진", "relation_type": "BETRAYED", 
             "strength": 9, "bidirectional": True},  # ERROR
            {"source": "서진", "target": "하나", "relation_type": "MENTOR",
             "strength": 7, "bidirectional": True}   # ERROR
        ]
    }
    return await consistency_check_node(state)

result = run_async(test_direction_conflict())
report = result.get("consistency_report", {})

print(f"\n" + "="*70)
print("🧭 DIRECTION_CONFLICT - final_value_candidate 구조")
print("="*70)

for c in report.get("conflicts", []):
    if c.get('type') == 'DIRECTION_CONFLICT':
        print(f"\n🟡 {c.get('description', '')[:60]}...")
        print(f"   suggested_action: {c.get('suggested_action')}")
        print(f"   \n   📦 final_value_candidate:")
        fvc = c.get('final_value_candidate', {})
        print(json.dumps(fvc, ensure_ascii=False, indent=6))

🧭 Direction Conflict 테스트 (final_value_candidate 확인)...
[CONSISTENCY] Validating: 3 chars, 0 events, 2 rels
[CONSISTENCY] Available names: {'서진', '하나', '이민호'}
[CONSISTENCY] Programmatic: DIRECTION_CONFLICT - Relationship BETRAYED from '이민호' to '서진' must be unidirectio...
[CONSISTENCY] Programmatic: DIRECTION_CONFLICT - Relationship MENTOR from '서진' to '하나' must be unidirectional...
[CONSISTENCY] Score: 50, HIGH: 0, MEDIUM: 5, LOW: 0
[CONSISTENCY] Auto-fix: 3 (ready_for_update: 3), Human review: 2
[CONSISTENCY] Requires re-extraction: True

🧭 DIRECTION_CONFLICT - final_value_candidate 구조

🟡 Relationship BETRAYED from '이민호' to '서진' must be unidirectio...
   suggested_action: AUTO_FIX
   
   📦 final_value_candidate:
{
      "table": "relationships",
      "key": {
            "source": "이민호",
            "target": "서진",
            "relation_type": "BETRAYED"
      },
      "update": {
            "bidirectional": false
      }
}

🟡 Relationship MENTOR from '서진' to '하나' must be unidirectio

## 2. 백엔드 AUTO_FIX 처리 시뮬레이션

In [4]:
print("="*70)
print("🔧 백엔드 AUTO_FIX 처리 시뮬레이션")
print("="*70)

# 충돌에서 AUTO_FIX 항목만 추출
auto_fix_conflicts = [
    c for c in report.get("conflicts", []) 
    if c.get("suggested_action") == SuggestedAction.AUTO_FIX
]

print(f"\nAUTO_FIX 가능한 충돌 수: {len(auto_fix_conflicts)}")

for conflict in auto_fix_conflicts:
    fvc = conflict.get("final_value_candidate", {})
    
    if fvc and "table" in fvc:
        table = fvc.get("table")
        key = fvc.get("key", {})
        update = fvc.get("update", {})
        
        # 시뮬레이션: SQL UPDATE 쿼리 생성
        print(f"\n📝 Generated UPDATE Query:")
        print(f"   UPDATE {table}")
        print(f"   SET {', '.join(f'{k} = {v}' for k, v in update.items())}")
        print(f"   WHERE source = '{key.get('source')}' AND target = '{key.get('target')}'")
        print(f"         AND relation_type = '{key.get('relation_type')}'")

🔧 백엔드 AUTO_FIX 처리 시뮬레이션

AUTO_FIX 가능한 충돌 수: 3

📝 Generated UPDATE Query:
   UPDATE relationships
   SET bidirectional = False
   WHERE source = '이민호' AND target = '서진'
         AND relation_type = 'BETRAYED'

📝 Generated UPDATE Query:
   UPDATE relationships
   SET bidirectional = False
   WHERE source = '서진' AND target = '하나'
         AND relation_type = 'MENTOR'


## 3. Trait 충돌 테스트 - 인간 선택 옵션 확인

In [5]:
async def test_trait_conflict():
    print("⚠️ Trait Conflict 테스트 (human selection options)...")
    state = create_base_state()
    state["extracted_characters"] = [
        {"name": "모순캐릭터", "role": "protagonist", "status": "alive",
         "personality": {"core_traits": ["coward", "brave"]}, "visual": {}}
    ]
    return await consistency_check_node(state)

trait_result = run_async(test_trait_conflict())
report = trait_result.get("consistency_report", {})

print(f"\n" + "="*70)
print("⚠️ CHARACTER_TRAIT_CONFLICT - final_value_candidate 구조")
print("="*70)

for c in report.get("conflicts", []):
    if c.get('type') == 'CHARACTER_TRAIT_CONFLICT':
        print(f"\n🔴 {c.get('description', '')[:60]}...")
        print(f"   suggested_action: {c.get('suggested_action')}")
        print(f"   \n   📦 final_value_candidate (인간 선택 필요):")
        fvc = c.get('final_value_candidate', {})
        print(json.dumps(fvc, ensure_ascii=False, indent=6))

print("\n💡 requires_selection: True → 백엔드는 인간에게 options 중 선택을 요청해야 함")

⚠️ Trait Conflict 테스트 (human selection options)...
[CONSISTENCY] Validating: 1 chars, 0 events, 2 rels
[CONSISTENCY] Available names: {'모순캐릭터'}
[CONSISTENCY] Programmatic: CHARACTER_TRAIT_CONFLICT - Character '모순캐릭터' has contradictory traits: 'coward' and 'br...
[CONSISTENCY] Programmatic: REFERENTIAL_INTEGRITY_ERROR - Relationship source '이민호' not found in character list....
[CONSISTENCY] Programmatic: REFERENTIAL_INTEGRITY_ERROR - Relationship target '서진' not found in character list....
[CONSISTENCY] Programmatic: REFERENTIAL_INTEGRITY_ERROR - Relationship source '서진' not found in character list....
[CONSISTENCY] Programmatic: REFERENTIAL_INTEGRITY_ERROR - Relationship target '하나' not found in character list....
[CONSISTENCY] Score: 0, HIGH: 6, MEDIUM: 1, LOW: 0
[CONSISTENCY] Auto-fix: 1 (ready_for_update: 1), Human review: 6
[CONSISTENCY] Requires re-extraction: True

⚠️ CHARACTER_TRAIT_CONFLICT - final_value_candidate 구조

🔴 Character '모순캐릭터' has contradictory core traits 'coward' a

## 4. 참조 무결성 오류 - 유사 이름 제안

In [6]:
async def test_referential_integrity():
    print("🔍 Referential Integrity 테스트 (suggestions 확인)...")
    state = create_base_state()
    state["relationship_graph"] = {
        "relationships": [
            {"source": "존재하지않음", "target": "서진", "relation_type": "FRIENDLY", 
             "strength": 5, "bidirectional": True}
        ]
    }
    return await consistency_check_node(state)

ref_result = run_async(test_referential_integrity())
report = ref_result.get("consistency_report", {})

print(f"\n" + "="*70)
print("🔍 REFERENTIAL_INTEGRITY_ERROR - final_value_candidate 구조")
print("="*70)

for c in report.get("conflicts", []):
    if c.get('type') == 'REFERENTIAL_INTEGRITY_ERROR':
        print(f"\n🔴 {c.get('description', '')}")
        print(f"   suggested_action: {c.get('suggested_action')}")
        print(f"   \n   📦 final_value_candidate:")
        fvc = c.get('final_value_candidate', {})
        print(json.dumps(fvc, ensure_ascii=False, indent=6))

print("\n💡 suggestions: 사용 가능한 캐릭터 이름 제안 → 인간이 선택하거나 새로 생성")

🔍 Referential Integrity 테스트 (suggestions 확인)...
[CONSISTENCY] Validating: 3 chars, 0 events, 1 rels
[CONSISTENCY] Available names: {'서진', '하나', '이민호'}
[CONSISTENCY] Programmatic: REFERENTIAL_INTEGRITY_ERROR - Relationship source '존재하지않음' not found in character list....
[CONSISTENCY] Score: 40, HIGH: 2, MEDIUM: 1, LOW: 0
[CONSISTENCY] Auto-fix: 1 (ready_for_update: 1), Human review: 2
[CONSISTENCY] Requires re-extraction: True

🔍 REFERENTIAL_INTEGRITY_ERROR - final_value_candidate 구조

🔴 Relationship source '존재하지않음' not found in character list.
   suggested_action: FLAG_FOR_HUMAN
   
   📦 final_value_candidate:
{
      "action": "choose_or_create",
      "invalid_name": "존재하지않음",
      "suggestions": [
            "서진",
            "하나",
            "이민호"
      ],
      "context": "relationship FRIENDLY to 서진"
}

💡 suggestions: 사용 가능한 캐릭터 이름 제안 → 인간이 선택하거나 새로 생성


## 5. Resolution Summary (ready_for_update 확인)

In [7]:
result = run_async(test_direction_conflict())
report = result.get("consistency_report", {})
resolution = report.get("resolution_summary", {})

print("="*70)
print("📊 Resolution Summary")
print("="*70)
print(f"\n총 충돌 수: {resolution.get('total_conflicts', 0)}")
print(f"\n🔧 AUTO_FIX (자동 수정 가능): {resolution.get('auto_fixable', 0)}")
print(f"   └─ 🆕 ready_for_update (바로 DB UPDATE 가능): {resolution.get('ready_for_update', 0)}")
print(f"👤 FLAG_FOR_HUMAN (인간 검토 필요): {resolution.get('needs_human_review', 0)}")
print(f"📝 OVERWRITE_WITH_NEW (자동 덮어쓰기): {resolution.get('auto_overwritable', 0)}")

🧭 Direction Conflict 테스트 (final_value_candidate 확인)...
[CONSISTENCY] Validating: 3 chars, 0 events, 2 rels
[CONSISTENCY] Available names: {'서진', '하나', '이민호'}
[CONSISTENCY] Programmatic: DIRECTION_CONFLICT - Relationship BETRAYED from '이민호' to '서진' must be unidirectio...
[CONSISTENCY] Programmatic: DIRECTION_CONFLICT - Relationship MENTOR from '서진' to '하나' must be unidirectional...
[CONSISTENCY] Score: 70, HIGH: 0, MEDIUM: 3, LOW: 0
[CONSISTENCY] Auto-fix: 3 (ready_for_update: 3), Human review: 2
[CONSISTENCY] Requires re-extraction: False
📊 Resolution Summary

총 충돌 수: 5

🔧 AUTO_FIX (자동 수정 가능): 3
   └─ 🆕 ready_for_update (바로 DB UPDATE 가능): 3
👤 FLAG_FOR_HUMAN (인간 검토 필요): 2
📝 OVERWRITE_WITH_NEW (자동 덮어쓰기): 0


## 6. Spring Boot Callback - 전체 구조

In [8]:
print("="*70)
print("📤 Spring Boot Callback 데이터 구조")
print("="*70)

# 첫 번째 충돌의 final_value_candidate만 표시
conflicts_sample = report.get("conflicts", [])[:2]
for c in conflicts_sample:
    if "final_value_candidate" in c:
        c["final_value_candidate"] = c["final_value_candidate"]  # Keep as-is

callback_data = {
    "consistency_report": {
        "overall_score": report.get("overall_score", 100),
        "requires_reextraction": report.get("requires_reextraction", False),
        "resolution_summary": report.get("resolution_summary", {}),
        "conflicts_sample": conflicts_sample,
        "neo4j_validation": report.get("neo4j_validation", {})
    }
}

print(json.dumps(callback_data, ensure_ascii=False, indent=2))

📤 Spring Boot Callback 데이터 구조
{
  "consistency_report": {
    "overall_score": 70,
    "requires_reextraction": false,
    "resolution_summary": {
      "auto_fixable": 3,
      "ready_for_update": 3,
      "needs_human_review": 2,
      "auto_overwritable": 0,
      "total_conflicts": 5
    },
    "conflicts_sample": [
      {
        "type": "RELATIONSHIP_CONFLICT",
        "description": "The BETRAYED relationship between 이민호 and 서진 should be unidirectional.",
        "severity": "MEDIUM",
        "affected_elements": [
          "이민호",
          "서진"
        ],
        "resolution_hint": "Set the BETRAYED relationship to be unidirectional.",
        "suggested_action": "AUTO_FIX",
        "final_value_candidate": {
          "bidirectional": false
        }
      },
      {
        "type": "DIALOGUE_CONSISTENCY_CONFLICT",
        "description": "Speech patterns for characters have not been provided, so cannot be validated.",
        "severity": "LOW-MEDIUM",
        "affected_eleme